In [1]:
from selenium import webdriver
import pandas as pd
import lxml.html
import csv


##自动打开edge，获取代码
driver = webdriver.Edge('C:\Program Files (x86)\Microsoft\Edge Beta\Application\edgedriver_win32\msedgedriver.exe')  ##这里是下载的edge驱动地址，可以自己去官网下驱动
driver.get('http://2018.cctv.com/medal_list/index.shtml')     ##数据来源央视2018平昌东奥会奖牌榜
content=driver.page_source  ##获取页面代码信息
driver.quit()   ##关闭edge

f = open('nation_data.csv', 'w' , encoding='utf-8')  ##打开csv文件选择utf-8格式，后续可能会有乱码现象
csv_writer = csv.writer(f)

##表头，因为网页上爬取的没有国家的英文拼写后续还要自己根据excel翻译和手动加上国家的英文名称
csv_writer.writerow((["排名", "国家", "金牌", "银牌", "铜牌", "总数"]))  
metree = lxml.html.etree ##正则很好，但是我用lxml
parser = metree.HTML(content)  ###content是网页的源码
td_list = parser.xpath("/html/body/div[3]/div/div/div/div/div[2]/div[3]/div/div/div/table/tbody[@id='medal_list1']//tr")  ##使用xpath定位
for td_item in td_list:
    num_item= td_item.xpath('.//text()')  ##读取内容信息
    csv_writer.writerow(num_item)  ##以行写入
f.close()  ##关闭文件

##接下来对爬取的csv文件手动解决乱码并去除空行添加英文名称，并转换为xlsx格式文件
##大概就是先用记事本打开将utf-8编码改为ansi，然后用excel表打开，使用ctrl+g去除多余的空行
##再将国家中文名使用百度翻译的api接口翻译成英文，对于一些不符合国际奥委会命名的翻译人工手动来改
##如果不好改，可以直接去github上下载数据集


In [12]:
from pyecharts import options as opts
from pyecharts.charts import Map
import pandas as pd
import os
datas=pd.read_excel('data_to_map.xlsx')     ##读取excel表，这里是自己手动将上面爬取存储csv文件转换成xlsx文件

# 基础数据
value = datas['总数']  ##奖牌总数转为列表
attr = datas['英文']  ##国家名称转为列表

# 绘图
c = (
    Map()
        .add("2018平昌冬奥会奖牌总数分布图", [list(z) for z in zip(attr, value)], "world")  ##添加标题与对应的国家、奖牌数信息
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))  ##不把所有国家的名称显示出来
        .set_global_opts(
        title_opts=opts.TitleOpts(title="2018平昌冬奥会奖牌总数分布图"),     ##左上方黑标题
        visualmap_opts=opts.VisualMapOpts(max_= 50),          ##调整最大参考值为50，因为获奖数最高的挪威奖牌数为39

    )
        .render(path="2018平昌冬奥会奖牌总数分布图.html")       ##保存为html
)

# 打开html
os.system("2018平昌冬奥会奖牌总数分布图.html")


0